In [1]:
import pandas as pd
import requests

def get_headers():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/90.0.4430.212 Safari/537.36'
    }
    return headers

class get_11v11:
    def __init__(self, date, venue = None):
        self.date = date
        self.venue = venue
        
        self.url = self.get_url()

        r = requests.get(self.url, headers = get_headers())

        self.tables = pd.read_html(r.content, flavor="bs4")
        
        self.table = self.get_table()
        self.pos = self.get_pos()

    def get_url(self):
        year = self.date[:4]
        month = pd.to_datetime(self.date).month_name().lower()
        day = self.date[8:]

        url = f"https://www.11v11.com/league-tables/league-two/{day}-{month}-{year}/"
        if self.venue and self.venue[0].upper() in ["H", "A"]:
            if self.venue[0].upper() == "H":
                self.venue = "home"
            else:
                self.venue = "away"
            url += f"{self.venue}"
        return url
    
    def get_table(self):
        table = self.tables[0]
        table.Pos = table.Pos.index + 1
        return table
    
    def get_pos(self):
        try:
            pos = self.table.query("Team.str.contains('Tranmere')").Pos.values[0]
        except:
            pos = "No table containing Tranmere Rovers found"
        return pos

In [47]:
class bbc_api:
    def __init__(self, date):
        self.date = date

        self.match_url = self.get_match_url()
        self.match_list = self.get_match_list()
        try:
            self.match_key = self.get_match_key()
            self.event_key = self.get_event_key()
            self.tournament_data = self.get_tournament_data()
            self.match_data = self.get_match_data()
            self.cup_data = self.get_cup_data()
            self.tranmere, self.opponent = self.get_teams()
            self.opp_name = self.match_data[self.opponent]["name"]["full"]
            self.teams = self.print_teams()
            self.goals_for, self.goals_against, self.score = self.get_score()

            self.lineup_url = self.get_lineup_url()
            self.lineup_data = self.get_lineup_data()
            self.tranmere_players = self.lineup_data["teams"][self.tranmere]["players"]
            self.opp_players = self.lineup_data["teams"][self.opponent]["players"]
            self.attendance = self.get_attendance()
            self.referee = self.get_referee()
            self.stadium = self.get_stadium()
            self.venue = self.get_venue()
            self.ko_time = self.get_ko_time()
            self.formation = self.get_formation()
        except Exception as e:
            return None

    def get_match_url(self):
        url = f"https://push.api.bbci.co.uk/data/bbc-morph-football-scores-match-list-data/endDate/{self.date}/startDate/{self.date}/team/tranmere-rovers/todayDate/{self.date}/version/2.4.6/withPlayerActions/true?timeout=5"
        return url
    
    def get_match_list(self):
        r = requests.get(self.match_url, headers = get_headers())
        match_list = r.json()

        if not match_list['matchData']:
            print(f"No matches found for {self.date}")
            return 
        else:
            return match_list
    
    def get_match_key(self):
        match_key = next(iter(self.match_list["matchData"][0]["tournamentDatesWithEvents"]))
        return match_key
    
    def get_event_key(self):
        event_key = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["events"][0]["eventKey"]
        return event_key

    def get_tournament_data(self):
        tournament_data = self.match_list["matchData"][0]["tournamentMeta"]
        return tournament_data    
    
    def get_cup_data(self):
        cup_data = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["round"]
        return cup_data

    def get_match_data(self):
        match_data = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["events"][0]
        return match_data
        
    def get_lineup_url(self):
        url = f"https://push.api.bbci.co.uk/data/bbc-morph-sport-football-team-lineups-data/event/{self.event_key}/version/1.0.8"
        return url
    
    def get_lineup_data(self):
        r = requests.get(self.lineup_url, headers = get_headers())
        lineup_data = r.json()
        return lineup_data
    
    def get_teams(self):
        if self.match_data["homeTeam"]["name"]["full"] == "Tranmere Rovers":
            tranmere = "homeTeam"
            opponent = "awayTeam"
        else:
            tranmere = "awayTeam"
            opponent = "homeTeam"
        return tranmere, opponent
    
    def print_teams(self):
        second_team = self.match_data[self.opponent]["name"]["full"]
        print(f"Tranmere are {self.tranmere}. {second_team} are {self.opponent}")

    def get_score(self):
        goals_for = self.match_data[self.tranmere]["scores"]["score"]
        goals_against = self.match_data[self.opponent]["scores"]["score"]
        score = f"{goals_for}-{goals_against}"
        return goals_for, goals_against, score
             
    def get_attendance(self):
        if "attendance" in self.lineup_data["meta"].keys():
            attendance = self.lineup_data["meta"]["attendance"].replace(",", "")
            return attendance
        else:
            print(f"No attendance data for {self.date}")
            return None
        
    def get_referee(self):
        if "referee" in self.lineup_data["meta"].keys():
            attendance = self.lineup_data["meta"]["referee"].replace(",", "")
            return attendance
        else:
            print(f"No referee for {self.date}")
            return None
    
    def get_formation(self):
        formation = self.lineup_data["teams"][self.tranmere]["formation"]
        formation = "-".join(str(formation))
        return formation
    
    def get_stadium(self):
        stadium = self.match_data["venue"]["name"]["full"]
        return stadium

    def get_venue(self):
        if match_data["venue"]["name"]["full"] == "Wembley Stadium":
            venue = "N"
        elif match_data["venue"]["name"]["full"] == "Prenton Park":
            venue = "H"
        elif match_data["venue"]["name"]["full"] != "Prenton Park":
            venue = "A"
        return venue
    
    def get_ko_time(self):
        ko_time = self.match_data["startTimeInUKHHMM"]
        return ko_time


In [81]:
date = "2023-08-08"

data = bbc_api(date)

match_data = bbc_api(date).match_data

Tranmere are awayTeam. Barnsley are homeTeam
Tranmere are awayTeam. Barnsley are homeTeam


In [82]:
class match_df:
    def __init__(self, date):
        self.date = date
        self.data = bbc_api(self.date)
        self.match_data = self.data.match_data
        self.players = self.data.tranmere_players

        self.goals = self.get_goals_df()
        self.player_apps = self.get_apps()
        self.cards = self.get_cards()
        self.yellow_cards = self.get_yellow_cards()
        self.red_cards = self.get_red_cards()

    def get_goals_df(self):
        player_actions = self.match_data[self.data.tranmere]["playerActions"]

        goals = []
        for player in player_actions:
            player_name = player["name"]["full"]    
            actions = player["actions"]
            for action in actions:
                if action["type"] == "goal":
                    goal = {
                        "game_date": self.date,
                        "player_name": player_name,
                        "goal_min": action["timeElapsed"],
                        "penalty": action["penalty"],
                        "own_goal": action["ownGoal"]
                    }
                    goals.append(goal)
            
        goals_df = pd.DataFrame(goals)
        return goals_df
    
    def get_apps(self):
        player_apps = []
        for player in self.players:
            player_name = player["name"]["full"]
            try:
                shirt_no = player["meta"]["uniformNumber"]
            except:
                shirt_no = None
            role = player["meta"]["status"].replace("bench", "sub")
            
            player_app = {
                "game_date": self.date,
                "player_name": player_name,
                "shirt_no": shirt_no,
                "role": role
            }
            player_apps.append(player_app)
        
        apps_df = pd.DataFrame(player_apps)
        return apps_df
    
    def get_cards(self):
        player_cards = []

        for player in self.players:
            player_name = player["name"]["full"]
            cards = player["bookings"]
            if cards:
                for card in cards:
                    player_card = {
                        "game_date": self.date,
                        "player_name": player_name,
                        "minute": card["timeElapsed"],
                        "card_type": card["type"]
                    }
                    player_cards.append(player_card)
        
        cards_df = pd.DataFrame(player_cards)
        return cards_df
    
    def get_yellow_cards(self):
        cards = self.cards
        yellows = cards.query("card_type == 'yellow-card'").rename(columns = {"minute": "min_yc"}).drop(columns = ["card_type"])
        if yellows.empty:
            print("No yellow cards")
            return None
        else:
            return yellows
    
    def get_red_cards(self):
        cards = self.cards
        reds = cards.query("card_type.str.contains('red')").rename(columns = {"minute": "min_so"}).drop(columns = ["card_type"])
        if reds.empty:
            print("No reds cards")
            return None
        else:
            return reds
        
match_df("2023-08-08").yellow_cards

Tranmere are awayTeam. Barnsley are homeTeam
No reds cards


,game_date,player_name,min_yc
0,2023-08-08,Jake Leake,69
1,2023-08-08,Paul Lewis,90


In [60]:
# player_actions = match_data[data.tranmere]["playerActions"]

# goals = []
# for player in player_actions:
#     player_name = player["name"]["full"]    
#     actions = player["actions"]
#     for action in actions:
#         if action["type"] == "goal":
#             goal = {
#                 "game_date": date,
#                 "player_name": player_name,
#                 "goal_min": action["timeElapsed"],
#                 "penalty": action["penalty"],
#                 "own_goal": action["ownGoal"]
#             }
#             goals.append(goal)

# goals_df = pd.DataFrame(goals)
# if not goals_df.empty:
#     goals_df.to_csv("data-updates/goals.csv", index = False)

# goals_df

In [ ]:
# players = data.tranmere_players

# player_apps = []
# player_cards = []
player_sub_mins = []
player_subs = []

for player in players:
        player_name = player["name"]["full"]
        try:
            shirt_no = player["meta"]["uniformNumber"]
        except:
            shirt_no = None
        role = player["meta"]["status"].replace("bench", "sub")
        
        # player_app = {
        #     "game_date": date,
        #     "player_name": player_name,
        #     "shirt_no": shirt_no,
        #     "role": role
        # }
        # player_apps.append(player_app)

        # cards = player["bookings"]
        # if cards:
        #     for card in cards:
        #         player_card = {
        #             "game_date": date,
        #             "player_name": player_name,
        #             "minute": card["timeElapsed"],
        #             "card_type": card["type"]
        #         }
        #         player_cards.append(player_card)

        subs = player["substitutions"]
        if subs:
            sub_min = subs[0]["timeElapsed"]
            try:
                sub_on_no = subs[0]["replacedBy"]["meta"]["uniformNumber"]
            except:
                sub_on_no = None
            player_on = subs[0]["replacedBy"]["name"]["full"]

            sub_mins_off = {
                "game_date": date,
                "player_name": player_name,
                "min_off": sub_min,
                "min_on": None
            }
            sub_mins_on = {
                "game_date": date,
                "player_name": player_on,
                "min_off": None,
                "min_on": sub_min
            }
            player_sub_mins.append(sub_mins_off)
            player_sub_mins.append(sub_mins_on)

            sub_on = {
                "game_date": date,
                "shirt_no": sub_on_no,
                "player_name": player_name,
                "on_for": shirt_no,
                "off_for": None
            }
            sub_off = {
                "game_date": date,
                "shirt_no": shirt_no,
                "player_name": player_on,
                "on_for": None,
                "off_for": sub_on_no
            }
            player_subs.append(sub_on)
            player_subs.append(sub_off)

# new_apps = pd.DataFrame(player_apps)
# new_subs = pd.DataFrame(player_subs)
new_sub_mins = pd.DataFrame(player_sub_mins)
new_cards = pd.DataFrame(player_cards)

new_yellows = new_cards[new_cards["card_type"] == "yellow-card"].rename(columns = {"minute": "min_yc"}).drop(columns = ["card_type"])
new_reds = new_cards[new_cards["card_type"].str.contains("red")].rename(columns = {"minute": "min_so"}).drop(columns = ["card_type"])

new_apps.to_csv("data-updates/player_apps.csv", index = False)
if not new_subs.empty:
    new_subs.to_csv("data-updates/subs.csv", index = False)
    new_sub_mins.to_csv("data-updates/sub_mins.csv", index = False)
if not new_cards.empty:
    new_yellows.to_csv("data-updates/yellow_cards.csv", index = False)
    new_reds.to_csv("data-updates/red_cards.csv", index = False)

In [123]:
import re

def get_game_type(data):
    if data.tournament_data["tournamentName"]["first"] in ["League One", "League Two", "National League"]:
        if "round" not in data.cup_data:
            return "League"
        else:
            if "Play-offs" in data.cup_data["round"]["full"]:
                return "League Play-Off"     
    else:
        return "Cup"

def get_cup_leg(match_data):
    if "leg" in match_data["eventType"]:
        return match_data["eventType"][:1]
    else:
        return None

def get_cup_name(cup_data):
    cup_name = cup_data["name"]
    if cup_name:
        cup_stage = cup_name["full"]
        if re.search(r"North(?:ern)?", cup_stage):
            cup_section = re.search(r"North(?:ern)?", cup_stage).group(0)
    else:
        cup_stage = None
        cup_section = None
    return cup_stage, cup_section

def get_cup_round(cup_stage):
    if cup_stage is None:
        return None
    elif " FINAL" in cup_stage.upper():
        return "F"
    elif cup_stage.upper() in ["PLAY-OFFS", "SEMI-FINALS"]:
        return "SF"
    elif "QUARTER-FINALS" in cup_stage.upper():
        return "QF"
    elif "FIFTH ROUND" in cup_stage.upper():
        return "5"
    elif "FOURTH ROUND" in cup_stage.upper():
        return "4"
    elif "THIRD ROUND" in cup_stage.upper():
        return "3"
    elif "SECOND ROUND" in cup_stage.upper():
        return "2"
    elif "FIRST ROUND" in cup_stage.upper():
        return "1"
    elif "GROUP" in cup_stage.upper():
        return "G"
    else:
        return None

def get_aet(match_data):
    if match_data["eventProgress"] == "EXTRATIMECOMPLETE":
        return 1
    else:
        return None

def get_shootout_outcome(pen_gf, pen_ga):
    if pen_gf:
        if pen_gf > pen_ga:
            pen_outcome = "W"
        elif pen_gf < pen_ga:
            return "L"
        pen_score = f"{pen_gf}-{pen_ga}"
    else:
        pen_outcome = None
        pen_score = None
    return pen_outcome, pen_score

def get_agg_outcome(agg_gf, agg_ga):
    if agg_gf:
        if agg_gf > agg_ga:
            agg_outcome = "W"
        elif agg_gf < agg_ga:
            agg_outcome = "L"
        agg_score = f"{agg_gf}-{agg_ga}"
    else:
        agg_outcome = None
        agg_score = None
    return agg_outcome, agg_score

def get_decider(match_data):
    decider = match_data["eventOutcomeType"]
    if decider == "shootout":
        return "pens"
    elif decider == "extra-time":
        return "extra time"
    else:
        return None

def get_cup_outcome(data, aet, pen_outcome, agg_outcome):
    if aet or pen_outcome or agg_outcome:
        return data.match_data[data.tranmere]["eventOutcome"].upper()[:1]
    else:
        return None

def get_outcome_desc(pen_outcome, pen_score, agg_outcome, agg_score):
    if pen_outcome:
        str_outcome = "Won" if pen_outcome == "W" else "Lost"
        if agg_outcome:
            outcome_desc = f"{agg_score}. {str_outcome} {pen_score} on pens"
        else:
            outcome_desc = f"{str_outcome} {pen_score} on pens"
    elif agg_outcome and not pen_outcome:
        str_outcome = "Won" if agg_outcome == "W" else "Lost"
        outcome_desc = f"{str_outcome} {agg_score} on agg"
    else:
        outcome_desc = None
    return outcome_desc

#########
date = "2023-08-08"
data = bbc_api(date)
match_data = data.match_data

#########

game_date = data.date
opposition = data.opp_name
venue = data.venue
goals_for = data.goals_for
goals_against = data.goals_against
score = data.score
goal_diff = goals_for - goals_against
game_type = get_game_type(data)
competition = data.tournament_data["tournamentName"]["full"].replace("Sky Bet ", "").replace("Vanarama", "")
#generic_comp
#league_tier
ko_time = data.ko_time

cup_leg = get_cup_leg(match_data)
cup_stage = get_cup_name(data.cup_data)[0]
cup_section = get_cup_name(data.cup_data)[1]
cup_round = get_cup_round(cup_stage)
aet = get_aet(match_data)

pen_gf = match_data[data.tranmere]["scores"]["shootout"]
pen_ga = match_data[data.opponent]["scores"]["shootout"]
pen_outcome = get_shootout_outcome(pen_gf, pen_ga)[0]
pen_score = get_shootout_outcome(pen_gf, pen_ga)[1]

agg_gf = match_data[data.tranmere]["scores"]["aggregate"]
agg_ga = match_data[data.opponent]["scores"]["aggregate"]
agg_outcome = get_agg_outcome(agg_gf, agg_ga)[0]
agg_score = get_agg_outcome(agg_gf, agg_ga)[1]

cup_outcome = get_cup_outcome(data, aet, pen_outcome, agg_outcome)
outcome_desc = get_outcome_desc(pen_outcome, pen_score, agg_outcome, agg_score)

game_length = 90 if aet is None else 120
stadium = data.stadium

# match_record = {
#     "game_date": game_date,
#     "opposition": opposition,
#     "venue": venue,
#     "goals_for": goals_for,
#     "goals_against": goals_against,
#     "score": score,
#     "outcome": outcome,
#     "goal_diff": goal_diff,
#     "game_type": game_type,
#     "competition": competition,
#     "ko_time": ko_time,
#     "cup_leg": cup_leg,
#     "cup_stage": cup_stage,
#     "cup_replay": cup_replay,
#     "cup_section": cup_section,
#     "aet": aet,
#     "pen_gf": pen_gf,
#     "pen_ga": pen_ga,
#     "pen_outcome": pen_outcome,
#     "pen_score": pen_score,
#     "agg_gf": agg_gf,
#     "agg_ga": agg_ga,
#     "agg_outcome": agg_outcome,
#     "agg_score": agg_score,
#     "decider": decider,
#     "cup_outcome": cup_outcome,
#     "outcome_desc": outcome_desc,
#     "game_length": game_length,
#     "stadium": stadium,
#     "event_id": event_id
# }

Tranmere are awayTeam. Barnsley are homeTeam
1


In [15]:
if data.match_list:
    print("Yes, data is there")
else:
    print("No, data is not there")

Yes, data is there


In [16]:
trfc_players = data.tranmere_players

for player in trfc_players:
    pos = player["meta"]["positionRegular"]
    print(pos)

goalkeeper
defender
defender
defender
defender
midfielder
midfielder
midfielder
forward
forward
forward
midfielder
forward
goalkeeper
forward
forward
defender
midfielder
